### Student Information
Name:劉宇凌

Student ID: 109136505

GitHub ID: wudishidove

Kaggle name: 水鬼不要QQ

Kaggle private scoreboard snapshot: https://imgur.com/a/bPRBHrw
<img src=" https://imgur.com/iFkFbqE.png" />

"[Snapshot](img/pic0.png)"
<img src="img/pic0.png" />

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2020-Lab2-Master Repo](https://github.com/fhcalderon87/DM2020-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2020-hw2-nthu/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the score (ie. 20% of 30% )

    - **Top 41% - 100%**: Get (101-x)% of the score, where x is your ranking in the leaderboard (ie. (101-x)% of 30% )   
    Submit your last submission __BEFORE the deadline (Dec. 5th 11:59 pm, Saturday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/fhcalderon87/DM2020-Lab1-Master/blob/master/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb), but make sure to fork the [DM2020-Lab2-Homework](https://github.com/fhcalderon87/DM2020-Lab2-Homework) repository this time! Also please __DON´T UPLOAD HUGE DOCUMENTS__, please use Git ignore for that.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 8th 11:59 pm, Tuesday)__. 

# Report of kaggle


## read data first

In [ ]:
import pandas as pd
import numpy as np
import json
#read data
df = pd.read_csv('data_identification.csv')
em = pd.read_csv('emotion.csv')
sample=pd.read_csv('sampleSubmission.csv')
usr= pd.read_json('tweets_DM.json',lines=True)

## check data long and divide into train data & test data 

In [ ]:
df.shape
train_x=df.loc[lambda a: a.identification=='train']
test_x=df.loc[lambda a: a.identification=='test']
usr.head()

## 將pd.read_json沒展開完全的資料展開

In [ ]:
#展開json中間的 tweet.hashtag text.......
test=pd.io.json.json_normalize(usr['_source'])

## get what data I want in json

In [ ]:
# joint two json dataframe
usr2=pd.concat([test[test.columns[1]],usr[usr.columns[0]]], axis=1)

usr2=pd.concat([usr2, test[test.columns[0]]], axis=1)

usr2=pd.concat([usr2, test[test.columns[2]]], axis=1)
usr2.head()

## rename index and joint the Data by user ID,
## then add emotion

In [ ]:
#rename the index
usr2=usr2.rename(columns={'tweet.tweet_id':'tweet_id'})
usr2.head()
#merge with train_user by ID 
final=pd.merge(train_x,usr2)
#check
final.head(20)
final.shape
# add motion
final2=pd.merge(final,em)
#check
final2.head()

In [ ]:
# same as test data
testData=pd.merge(test_x,usr2)
testData.head()

## transform the text into feature by TFIDF
transform feature by word based and character based
then stack two transform features

In [ ]:
from scipy.sparse import hstack
from sklearn import preprocessing, model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, make_scorer, roc_curve, roc_auc_score

In [ ]:
# word based
tfv_word = TfidfVectorizer(min_df=150,  max_features= 100000,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,5),stop_words = 'english')
tfv_word.fit(final2['tweet.text'])
train_tfv_word =  tfv_word.transform(final2['tweet.text'])
test_tfv_word = tfv_word.transform(testData['tweet.text'])
# character based
tfv_char = TfidfVectorizer(sublinear_tf=True,strip_accents='unicode',analyzer='char',stop_words='english',ngram_range=(2, 6),max_features=100000)
tfv_char.fit(final2['tweet.text'])
train_tfv_char = tfv_char.transform(final2['tweet.text'])
test_tfv_char = tfv_char.transform(testData['tweet.text'])


In [ ]:
# 疊起來
train_tfidf = hstack([train_tfv_word, train_tfv_char])
test_tfidf = hstack([test_tfv_word, test_tfv_char])

In [ ]:
# logistic Regression
y = final2['emotion']
lr = LogisticRegression()
lr.fit(train_tfidf, y)
tf_predict = lr.predict(test_tfidf)

In [ ]:
#change datatype from array to pd
tf_predict2=pd.DataFrame(tf_predict)
tf_predict2.head()
temp=tf_predict2
#temp.head()
#add id into output
output=pd.concat([testData['tweet_id'], temp], axis=1)
#output.head()

#rename
output=output.rename(columns={'tweet_id':'id'})
#output[lambda a: a.'0'=='emotion']
output=output.rename(columns={0:'emotion'})
output.head()

#saveFile
output.to_csv('preditOutput6.csv', index=False )

## what i found and what else i try 
### i found text transform feature increase can improve the accuracy
### below cell is what i try in tfidf

In [ ]:
#max_features= 10000
#max_features= 25000
#max_features= 40000
#max_features= 50000
#max_features= 100000

### and i found increase ngram can improve accuracy too
i change ngram from (1,2) to (1,6) and feature max=50000
i get 0.488 from 0.46 at the same 50000 feature.

### else method include bag of word(iltk) and dicition tree 
## i found dicition tree will overfitting and get bad accuracy(0.2) on test

-------------------------------
# part 1

---
### ** >>> Exercise 1 (Take home): **  
Plot word frequency for Top 30 words in both train and test dataset. (Hint: refer to DM lab 1)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#count vector
train_count = CountVectorizer(stop_words = 'english')
test_count = CountVectorizer(stop_words = 'english')
X_train_counts = train_count.fit_transform(train_df.text)
X_test_counts = test_count.fit_transform(test_df.text)
print('train shape:', X_train_counts.shape)
print('test shape:', X_test_counts.shape)

#to pd
X_train_count_pd = pd.DataFrame(columns = train_count_vect.get_feature_names(), data = X_train_counts.toarray())
X_test_count_pd = pd.DataFrame(columns = test_count_vect.get_feature_names(), data = X_test_counts.toarray())

#sort 30
test_30 = X_test_count_pd.sum().sort_values(ascending=False)[:30]
train_30 = X_train_count_pd.sum().sort_values(ascending=False)[:30]

import matplotlib.pyplot as plt
#show picture
plt.figure(figsize=(30,10))
plt.bar(train_30.index, train_30, label = 'word frequencies', align = 'edge')
plt.legend()


"[Snapshot](img/pic1.png)"
<img src="img/pic1.png" />

---
### ** >>> Exercise 2 (Take home): **  
Generate an embedding using the TF-IDF vectorizer instead of th BOW one with 1000 features and show the feature names for features [100:110].

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer(max_features=1000)
tfidf.fit(train_df['text'])
#get name
feature_names = tfidf.get_feature_names()
feature_names[100:110]

"[Snapshot](img/pic2.png)"
<img src="img/pic2.png" />

---
### ** >>> Exercise 3 (Take home): **  
# Can you interpret the results above? What do they mean?

---
A: training accuracy: 0.99,testing accuracy: 0.66. so it is overfitting.

And in the confution matrix, anger has the best accuracy.

---
### ** >>> Exercise 4 (Take home): **  
Build a model using a ```Naive Bayes``` model and train it. What are the testing results? 

*Reference*: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
# Answer here
from sklearn.naive_bayes import MultinomialNB
NBmodel = MultinomialNB()
NBmodel.fit(X_train, y_train)
print('train accuracy:', NBmodel.score(X_train, y_train))
print('test accuracy:', NBmodel.score(X_test, y_test))
y_test_pred = NBmodel.predict(X_test)

# plot your confusion matrix
cm2 = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
my_tags = ['anger', 'fear', 'joy', 'sadness']
plot_confusion_matrix(cm2, classes=my_tags, title='Confusion matrix')

"[Snapshot](img/pic3.png)"
<img src="img/pic3.png" style="width: 300px;"/>
---
### ** >>> Exercise 5 (Take home): **  

How do the results from the Naive Bayes model and the Decision Tree model compare? How do you interpret these differences? Use the theoretical background covered in class to try and explain these differences.

---
A:decition tree is overfitting, Naive has a lower training score,but maybe can increase its acuuracy by bigger data.

---
---
### ** >>> Exercise 6 (Take home): **  

Plot the Training and Validation Accuracy and Loss (different plots), just like the images below (Note: the pictures below are an example from a different model). How to interpret the graphs you got? How are they related to the concept of overfitting/underfitting covered in class?
<table><tr>
    <td><img src="pics/pic3.png" style="width: 300px;"/> </td>
    <td><img src="pics/pic4.png" style="width: 300px;"/> </td>
</tr></table>

In [ ]:
# Answer here
# accuracy
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.plot(training_log.epoch, training_log.accuracy, training_log.epoch, training_log.val_accuracy)
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend(["training accuracy", "val_accuracy"])
plt.title("Training accuracy per epoch")

# loss
plt.subplot(1, 2, 2)
plt.plot(training_log.epoch, training_log.loss, training_log.epoch, training_log.val_loss)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(["training loss", "val_loss"])
plt.title("Training loss per epoch")
plt.show()

<img src="img/pic4.png" />

---
### ** >>> Exercise 7 (Take home): **  

Now, we have the word vectors, but our input data is a sequence of words (or say sentence). 
How can we utilize these "word" vectors to represent the sentence data and train our model?

---
A: use like k-means or other Neurol network classify model

---
### ** >>> Exercise 8 (Take home): **  

Generate a t-SNE visualization to show the 15 words most related to the words "angry", "happy", "sad", "fear" (60 words total).

In [ ]:
# Answer here
from sklearn.manifold import TSNE
word_list = ['happy', 'angry', 'data', 'mining']


topn = 15
angry_words = ['angry'] + [word_ for word_, sim_ in w2v_google_model.most_similar('angry', topn=topn)]        
happy_words = ['happy'] + [word_ for word_, sim_ in w2v_google_model.most_similar('happy', topn=topn)]
sad_words = ['sad'] + [word_ for word_, sim_ in w2v_google_model.most_similar('sad', topn=topn)]        
fear_words = ['fear'] + [word_ for word_, sim_ in w2v_google_model.most_similar('fear', topn=topn)]

target_words = happy_words + angry_words + sad_words + fear_words

cn = topn + 1
color = ['b'] * cn + ['g'] * cn + ['r'] * cn + ['y'] * cn

## w2v model
model = w2v_google_model

## prepare training word vectors
size = 200
target_size = len(target_words)
all_word = list(model.vocab.keys())
word_train = target_words + all_word[:size]
X_train = model[word_train]

## t-SNE model
tsne = TSNE(n_components=2, metric='cosine', random_state=28)

## training
X_tsne = tsne.fit_transform(X_train)

## show picture
plt.figure(figsize=(7.5, 7.5), dpi=115)
plt.scatter(X_tsne[:target_size, 0], X_tsne[:target_size, 1], c=color)
for label, x, y in zip(target_words, X_tsne[:target_size, 0], X_tsne[:target_size, 1]):
    plt.annotate(label, xy=(x,y), xytext=(0,0),  textcoords='offset points')
plt.show()

<img src="img/pic5.png" />